In [ ]:
import random
import math

def is_max_die_value(dice_str, roll_value):
    num_dice, num_faces = map(int, dice_str.split('d'))
    return roll_value == num_faces    

def roll_dice(dice_str, explode, advantage=0):
    try:
        # Split the input string into number of dice and number of faces
        num_dice, num_faces = map(int, dice_str.split('d'))

        # Roll the dice and return the results
        results = []

        for roll_num in range(num_dice):
            # Roll a die with the given number of faces

            if roll_num == 0:
                num_dice = abs(advantage) + 1
                result_list = [random.randint(1, num_faces) for _ in range(num_dice)]

                if advantage > 0:
                    result = max(result_list)
                elif advantage < 0:
                    result = min(result_list)
                else:
                    result = result_list[0]

                # If explode is True and the result equals the number of faces, keep rolling
                while explode and result == num_faces:
                    results.append(result)  # Record the result
                    result = random.randint(1, num_faces)
            else:
                result = random.randint(1, num_faces)

            results.append(result)  # Record the final result

        return results
    
    except ValueError:
        # Handle the case where the input values are not valid
        return "Invalid input values. Please use integers for num_dice and num_faces."
    
def attack_roll(dice_str, damage_threshold, damage_modifier, advantage):
    # Roll the dice using the roll_dice function
    dice_results = roll_dice(dice_str, True, advantage)

    # Check if the first value is less than the damage threshold
    if dice_results[0] <= damage_threshold:
        return 0  # No damage done

    # Initialize total damage with the sum of dice results and the damage modifier
    total_damage = sum(dice_results) + damage_modifier

    return total_damage

# Example usage:
print(roll_dice("2d4", True, 3))

damage = attack_roll("1d4", damage_threshold=1, damage_modifier=5, advantage=0)
print(damage)

In [ ]:
def get_average_damage(dice_str, advantage, num_tests=100):
    average = 0
    for _ in range(num_tests):
        damage = attack_roll(dice_str, 1, 0, advantage)
        average += damage / num_tests
    return average

print(get_average_damage("1d4", 0))

In [ ]:
class Character:
    def __init__(self, name, hp, ac, num_attacks, attack_dice, damage_modifier):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.num_attacks = num_attacks
        self.attack_dice = attack_dice
        self.damage_modifier = damage_modifier

In [ ]:
fighter = Character(name="Fighter 3", hp=40, ac=18, num_attacks=1, attack_dice="1d8", damage_modifier=3)
monster = Character(name="Light Armor Monster 3", hp=34, ac=12, num_attacks=2, attack_dice="1d10", damage_modifier=4)

In [ ]:
class AttackProcessor:
    def perform_attacks(self, attacker, defender, defense_processor, print_results = False):
        # ABSTRACT DEFINITION FOR PERFORMING ATTACKS
        pass

class DefenseProcessor:
    def start_of_round(self):
        # PROCESSING AT START OF ROUND
        pass
        
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        # ABSTRACT DEFINITION FOR CHECKING IF AN ATTACK HIT
        pass
        
    def defend_against_attack(self, defender, attacker, die_rolls, print_results = False):
        # ABSTRACT DEFINITION FOR DEFENDING ATTACKS
        pass

In [ ]:
def get_damage_for_round(attacker, defender, attack_processor, defense_processor, print_results=False):
    defense_processor.start_of_round()
    damage = attack_processor.perform_attacks(attacker, defender, defense_processor, print_results)
    return damage

In [ ]:
class NimbleFighterAttack(AttackProcessor):
    def perform_attacks(self, attacker, defender, defense_processor, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, 0, print_results)
            total_damage += damage

        # Process one more attack with disadvantage
        damage = self.perform_single_attack(attacker, defender, defense_processor, -1, print_results)
        total_damage += damage
            
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, advantage, print_results=False):
        die_rolls = roll_dice(attacker.attack_dice, True, advantage)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attacker.attack_dice}+{attacker.damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        if hit:
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [ ]:
class NimbleMonsterLightArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, print_results = False):
        damage = sum(die_rolls) + attacker.damage_modifier
        return damage
        
print(get_damage_for_round(fighter, monster, NimbleFighterAttack(), NimbleMonsterLightArmorDefense(), print_results=True))

In [ ]:
class NimbleMonsterMediumArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, print_results = False):
        damage = sum(die_rolls)

        is_crit = is_max_die_value(attacker.attack_dice, die_rolls[0])
        if is_crit:
            damage += attacker.damage_modifier
        
        return damage

print(get_damage_for_round(fighter, monster, NimbleFighterAttack(), NimbleMonsterMediumArmorDefense(), print_results=True))

In [ ]:
class NimbleMonsterHeavyArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, print_results = False):
        damage = sum(die_rolls) + attacker.damage_modifier
        
        is_crit = is_max_die_value(attacker.attack_dice, die_rolls[0])
        if not is_crit:
            damage = math.ceil(damage / 2)
        
        return damage
        
print(get_damage_for_round(fighter, monster, NimbleFighterAttack(), NimbleMonsterHeavyArmorDefense(), print_results=True))

In [ ]:
class NimbleMonsterArmorDefense(DefenseProcessor):
    def __init__(self, monster):
        if monster.ac < 14:
            self.armor_processor = NimbleMonsterLightArmorDefense()
        elif monster.ac < 18:
            self.armor_processor = NimbleMonsterMediumArmorDefense()
        else:
            self.armor_processor = NimbleMonsterHeavyArmorDefense()

    def start_of_round(self):
        self.armor_processor.start_of_round()

    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        return self.armor_processor.did_attack_hit(attacker, defender, attack_dice, print_results)

    def defend_against_attack(self, defender, attacker, die_rolls, print_results = False):
        return self.armor_processor.defend_against_attack(defender, attacker, die_rolls, print_results)

monster.ac = 12
print(get_damage_for_round(fighter, monster, NimbleFighterAttack(), NimbleMonsterArmorDefense(monster), print_results=True))
print()

monster.ac = 15
print(get_damage_for_round(fighter, monster, NimbleFighterAttack(), NimbleMonsterArmorDefense(monster), print_results=True))
print()

monster.ac = 19
print(get_damage_for_round(fighter, monster, NimbleFighterAttack(), NimbleMonsterArmorDefense(monster), print_results=True))

In [ ]:
class NimbleMonsterAttack(AttackProcessor):
    def perform_attacks(self, attacker, defender, defense_processor, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, 0, print_results)
            total_damage += damage
            
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, advantage, print_results=False):
        die_rolls = roll_dice(attacker.attack_dice, True, advantage)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attacker.attack_dice}+{attacker.damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        if hit:
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [ ]:
class NimblePlayerDefense(DefenseProcessor):
    def __init__(self):
        self.has_blocked = False

    def start_of_round(self):
        self.has_blocked = False

    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, print_results = False):
        damage = sum(die_rolls) + attacker.damage_modifier
        
        if not self.has_blocked:
            damage_reduction = defender.ac - 8

            if print_results:
                print(f'{defender.name} blocks {damage_reduction} damage')
            
            damage = max(0, damage - damage_reduction)
            self.has_blocked = True
        
        return damage

print(get_damage_for_round(monster, fighter, NimbleMonsterAttack(), NimblePlayerDefense(), print_results=True))

In [ ]:
def get_dpr(attacker, defender, attack_processor, defense_processor, num_simulations=100000):
    average_damage = 0.0
    for _ in range(num_simulations):
        damage = get_damage_for_round(attacker, defender, attack_processor, defense_processor)
        average_damage += damage / num_simulations
    return average_damage

In [ ]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Character(name="Fighter 3", hp=40, ac=18, num_attacks=1, attack_dice="1d8", damage_modifier=3)
monster = Character(name="Monster", hp=34, ac=12, num_attacks=2, attack_dice="1d10", damage_modifier=4)
# END PLAYER AND MONSTER DEFINITIONS

player_dpr = get_dpr(player, monster, NimbleFighterAttack(), NimbleMonsterArmorDefense(monster))
player_turns = monster.hp / player_dpr
monster_dpr = get_dpr(monster, player, NimbleMonsterAttack(), NimblePlayerDefense())
monster_turns = player.hp / monster_dpr

print("Player DPR: " + str(player_dpr))
print("Monster DPR: " + str(monster_dpr))
print("Player Turns to Win: " + str(player_turns))
print("Monster Turns to Win: " + str(monster_turns))
